# ⚠️ PLANNER NOT YET IMPLEMENTED IN PYTHON

# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [ ]:
from init import *
import semantic_kernel as sk

kernel = sk.KernelBuilder.create_kernel()

useAzureOpenAI = False
model = "text-davinci-002"

# Configure AI backend used by the kernel
if (useAzureOpenAI):
    api_key, endpoint = azure_openai_settings_from_dot_env()
    kernel.config.add_azure_openai_completion_backend("davinci", model, endpoint, api_key)
else:
    api_key, org_id = openai_settings_from_dot_env()
    kernel.config.add_openai_completion_backend("davinci", model, api_key, org_id)

### Setting Up the Planner
The planner is located in the Semantic Kernel's CoreSkills and requires Orchestration

In [ ]:
from semantic_kernel.core_skills import PlannerSkill
from semantic_kernel.kernel_extensions.import_semantic_skill_from_directory import import_semantic_skill_from_directory

# Load native skill into the kernel registry, sharing its functions with prompt templates
planner = kernel.import_skill(PlannerSkill(kernel))

You can see that the Planner took my ask and converted it into an XML-based plan detailing
how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk.

In [ ]:
skills_directory = "../../skills"
skill = import_semantic_skill_from_directory(kernel, skills_directory, "SummarizeSkill")
skill = import_semantic_skill_from_directory(kernel, skills_directory, "WriterSkill")

Define your ASK. What do you want the Kernel to do?

In [ ]:
ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other.";
originalPlan = await kernel.run_on_str_async(ask, planner["CreatePlan"])

print("Original plan:\n");
print(originalPlan.variales.to_plan().plan_string);

As you can see in the above plan, the AI has determined which functions to call 
in order to fulfill the user ask. The output of each step of the plan gets set as `setContextVariable` which makes it available as `input` to the next skill. 

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [ ]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.CreateSemanticFunction(sk_prompt, "shakespeare", "ShakespeareSkill", max_tokens: 2000, temperature: 0.2, top_p: 0.5);

Let's update our ask using this new skill.

In [ ]:
ask = """Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other"""

In [ ]:
new_plan = await kernel.run_on_str_async(ask, planner["CreatePlan"]);

In [ ]:
print("Updated plan:\n");
print(new_plan.variales.to_plan().plan_string);

### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a skill called `ExecutePlan`.

In [ ]:
execution_results = new_plan

In [ ]:
step = 1
maxSteps = 10
while (not execution_results.variales.to_plan().is_complete and step < maxSteps):
    results = await kernel.run_on_str_async(execution_results.variables, planner["ExecutePlan"])
    if (results.variabes.to_plan().is_successful):
        print("Step {step} - Execution results:\n")
        print(results.variables.to_plan().plan_string)

        if (results.variables.to_plan().is_complete):
            print("Step {step} - COMPLETE!")
            print(results.variables.to_plan().result)
            break
    else:
        print("Step {step} - Execution failed:")
        print(results.variables.to_plan().result)
        break
    
    execution_results = results
    step += 1
    print("")